# Assignment 4

In [167]:
import numpy as np
import pandas as pd 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from itertools import chain
from numpy import pi, cos, sin, exp
from scipy.signal import correlate, square, unit_impulse, welch, periodogram, butter, lfilter
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

### Import data

In [2]:
# Import data
D1 = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=1, usecols=[0,35])

D2 = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/UVicSci_temperature.dat', 
                 header=2)

In [3]:
# Convert time in D1 from MatLab time to Python Time
D1['Time'] = D1['NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
D12 = D1.rename(index=str, 
                columns={"NaN": "MatLab Time", "48.4623": "Temperature"})

In [4]:
# Reorder columns 
cols = D12.columns.tolist()
cols = cols[-1:] + cols[:-1]
D123 = D12[cols]

# Set time as index column
DH = D123.set_index('Time')

In [5]:
# Generate dates for D2 (minute resolution data)
date = pd.date_range(start='2011-12-31 17:00:00.000000', 
                     end='2017-08-30 16:59:00.000000', 
                     freq='min')

# Insert dates into D1 dataframe
D2.insert(loc=0, column='Time', value=date)

# Rename the columns
D22 = D2.rename(index=str, columns={"2979360": "Temperature"})

# Set index
DM = D22.set_index('Time')

In [74]:
# Select the dates:
# Hour resolution data
DH1 = DH.loc['2015-12-01 00:00':'2016-03-01 23:00']['Temperature']
DH2 = DH.loc['2016-06-01 00:00':'2016-09-01 23:00']['Temperature']

# Minuite resolution data
DM1 = DM.loc['2015-12-01 00:00':'2016-03-01 23:59']['Temperature']
DM2 = DM.loc['2016-06-01 00:00':'2016-09-01 23:59']['Temperature']

### Relevant functions 

In [75]:
# Inverse chi-squared distribution
df = 2
CI = 0.95
bounds = [(1-CI)/2, 1-(1-CI)/2]

chiT = df / (2 * sci.stats.chi2.ppf(bounds[0], df))
chiB = df / (2 * sci.stats.chi2.ppf(bounds[1], df))

## Question 1: Power spectral density

### a) Hour resolution: Power spectral density $S_{yy}$ based on periodgrams using Hanning window with $50\%$ overlap

In [154]:
steps_a = len(DH1)
# dTa = 3600
# FSa = np.fft.rfftfreq(steps_a, dTa)

In [155]:
# Generate the power spectral density 
F1a, G1a = periodogram(DH1, 1/3600, \
            window=sci.signal.windows.hann(int(steps_a)),\
            nfft = steps_a, detrend=False,\
            return_onesided=True, scaling = 'density')

F2a, G2a = periodogram(DH2, 1/3600, \
            window=sci.signal.windows.hann(int(steps_a)),\
            nfft = steps_a, detrend=False,\
            return_onesided=True, scaling = 'density')

# scipy.signal.periodogram(x, fs=1.0, window=None, nfft=None, detrend='constant', return_onesided=True, scaling='density', axis=-1)

In [156]:
# Generate the confidence limits 
G1Ta = G1a * chiT * 4/3
G1Ba = G1a * chiB * 4/3

G2Ta = G2a * chiT * 4/3
G2Ba = G2a * chiB * 4/3

In [157]:
# Plot it out 
fig, (ax1a, ax2a) = plt.subplots(2, 1, figsize=(10, 8))

title1a = r'Power spectral density of UVicSCI temperature data from 1 Dec. 2015 to 1 Mar. 2016 using the Hanning window with $95\%$ confidence intervals. Hour resolution.'
ax1a.loglog(F1a, G1Ta, 'b--', F1a, G1Ba, 'b--', alpha = 0.5)
ax1a.loglog(F1a, G1a)
ax1a.set_title("\n".join(wrap(title1a, 90)))
ax1a.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
ax1a.set_xlabel(r'Frequency [$Hz$]')

title2a = r'Power spectral density of UVicSCI temperature data from 1 June 2016 to 1 Sept. 2016 using the Hanning window with $95\%$ confidence intervals. Hour resolution.'
ax2a.loglog(F2a, G2Ta, 'b--', F2a, G2Ba, 'b--', alpha = 0.5)
ax2a.loglog(F2a, G2a)
ax2a.set_title("\n".join(wrap(title2a, 90)))
ax2a.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
ax2a.set_xlabel(r'Frequency [$Hz$]')

fig.tight_layout()
plt.show()

<Figure size 720x576 with 2 Axes>

### b) Plot power spectral density of summer and winter data on same figure in variance preserving form 

### c) Minuite resolution: Power spectral density $S_{yy}$ based on periodgrams using Hanning window with $50\%$ overlap

In [124]:
steps_c = int(len(DM1)/4)
# dTc = 60
# FSc = np.fft.rfftfreq(steps_c, dTc)

In [125]:
# Generate the power spectral density 
F1c, G1c = periodogram(DM1, 1/60, \
            window=sci.signal.windows.hann(int(steps_c)),\
            nfft = steps_c, detrend=False,\
            return_onesided=True, scaling = 'density')

F2c, G2c = periodogram(DM2, 1/60, \
            window=sci.signal.windows.hann(int(steps_c)),\
            nfft = steps_c, detrend=False,\
            return_onesided=True, scaling = 'density')

In [126]:
# Generate the confidence limits 
G1Tc = G1c * chiT * 4/3
G1Bc = G1c * chiB * 4/3

G2Tc = G2c * chiT * 4/3
G2Bc = G2c * chiB * 4/3

In [153]:
# Plot it out 
fig, (ax1c, ax2c) = plt.subplots(2, 1, figsize=(10, 8))

title1c = r'Power spectral density of UVicSCI temperature data from 1 Dec. 2015 to 1 Mar. 2016 using the Hanning window with $95\%$ confidence intervals. Minuite resolution.'
ax1c.loglog(F1c, G1Tc, 'b--', F1c, G1Bc, 'b--', alpha = 0.5)
ax1c.loglog(F1c, G1c)
ax1c.set_title("\n".join(wrap(title1c, 90)))
ax1c.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
ax1c.set_xlabel(r'Frequency [$Hz$]')

title2c = r'Power spectral density of UVicSCI temperature data from 1 June 2016 to 1 Sept. 2016 using the Hanning window with $95\%$ confidence intervals. Minuite resolution.'
ax2c.loglog(F2c, G2Tc, 'b--', FSc, G2Bc, 'b--', alpha = 0.5)
ax2c.loglog(F2c, G2c)
ax2c.set_title("\n".join(wrap(title2c, 90)))
ax2c.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
ax2c.set_xlabel(r'Frequency [$Hz$]')

fig.tight_layout()
plt.show()

<Figure size 720x576 with 2 Axes>

# Question 2: Filter design

# Question 3: PSD and filtering of synthetic data

$$x(t) = \cos(22 \pi t) + 0.7\sin(14 \pi t) + 0.5\sin(147 \pi t)$$


In [197]:
steps3 = 3*10**4
t3 = np.linspace(-15, 15, steps3, endpoint=False)

In [198]:
# Define x(t)
x1 = cos(22 * pi * t3)
x2 = 0.7 * sin(14 * pi * t3)
x3 = 0.5 * sin(147 * pi * t3)

x = x1 + x2 + x3

In [200]:
# Plot it out 
plt.figure(figsize=(10, 4))
plt.plot(t3, x)
plt.title(r'$x(t)$ from $t=-1$ to $t=1$')
plt.xlabel(r'Time ($s$)')
plt.ylabel(r'$x(t)$')
plt.xlim(-1, 1)

plt.show()

<Figure size 720x288 with 1 Axes>

In [201]:
# Get the power spectral density 
Fx, Gx = welch(x, steps3/2, nperseg=steps3,\
            window=sci.signal.windows.hann(int(steps3)),\
            noverlap = steps3/2, nfft = steps3, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

In [202]:
# Plot it out 
plt.figure(figsize=(10, 4))
plt.loglog(Fx, Gx)
plt.title(r'Power density spectrum of $x(t)$')
plt.xlabel(r'Frequency ($Hz$)')
plt.ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')

plt.show()

<Figure size 720x288 with 1 Axes>

In [358]:
# Design the filter
def bandpass(x, CutL, CutH, fs, order):
    fn = fs/2
    low = CutL / fn
    high = CutH / fn
    b, a = butter(order, [low, high], btype='bandpass')
    y = lfilter(b, a, x)
    return y

In [359]:
# Butter up x(t)
CutL = 10
CutH = 28
y1 = bandpass(x, CutL, CutH, steps3/(2*15), 6)

plt.figure(figsize=(10, 4))
plt.plot(t3, x1, label=r'$x_1$')
plt.plot(t3, y1, label=r'$|H(\omega)|$ (Frequency responce of filter)')
title3b = r'Isolation of $x1=\cos(22\pi t)$ signal from x(t) via a Butterworth filter with cut-off frequencies at {0} $rad/s$ and {1} $rad/s$'.format(int(CutL), int(CutH))
plt.title("\n".join(wrap(title3b, 90)))
plt.xlabel(r'Time ($s$)')
plt.ylabel(r'$|H(\omega)|$')

plt.legend(loc=2)
plt.xlim(-1, 1)

(-1, 1)

<Figure size 720x288 with 1 Axes>

In [349]:
# Power spectra of the filtered signal 
Fy1, Gy1 = welch(y1, steps3/2, nperseg=steps3,\
            window=sci.signal.windows.hann(int(steps3)),\
            noverlap = steps3/2, nfft = steps3, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

In [353]:
# Plot it out 
plt.figure(figsize=(10, 4))
plt.loglog(Fx, Gx, label = r'PDS of $x(t)$')
plt.loglog(Fy1, Gy1, label = r'PDS of $|H(\omega)|$')
plt.title(r'Power density spectrum of $x(t)$ and $|H(\omega)|$')
plt.xlabel(r'Frequency ($Hz$)')
plt.ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
plt.legend()

plt.show()

<Figure size 720x288 with 1 Axes>

In [365]:
# Plot differnce between recoverd and actual signal 
plt.figure(figsize=(10, 4))
plt.plot(t3, y1-x1)
title3b = r'Difference between $x1=\cos(22\pi t)$ and the isolated signal $x_f(t)$ via a Butterworth filter with cut-off frequencies at {0} $rad/s$ and {1} $rad/s$'.format(int(CutL), int(CutH))
plt.title("\n".join(wrap(title3b, 90)))
plt.xlabel(r'Time ($s$)')
plt.ylabel(r'$x_f(t) - x_1(t)$')
plt.show()

<Figure size 720x288 with 1 Axes>